In [1]:
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import shapely
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon
from sqlalchemy import text

plt.style.use("ggplot")

# Høyanger: Add catchment data to PostGIS

Add catchment data from the template here:

    critical_loads_2/hoyanger/hoyanger_stns.xlsx

In [2]:
# Connect to PostGIS
eng = nivapy.da.connect_postgis(admin=True)

Connection successful.


## 1. Add project

In [3]:
# Read projects
xl_path = r"../data/hoyanger_stns.xlsx"
proj_df = pd.read_excel(xl_path, sheet_name="project")
assert len(proj_df) == 1

proj_df

,project_code,project_name,aquamonitor_id,contact,description
0,220263,Høyanger,NaN,KAU,Critical loads calculations for Høyanger


In [4]:
# # Add project
# proj_df.to_sql(
#    "projects", schema="niva", con=eng, if_exists="append", index=False,
# )

## 2. Add stations

In [5]:
# Read stations
xl_path = r"../data/hoyanger_stns.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="stations")
stn_df["longitude"] = stn_df["longitude"].round(6)
stn_df["latitude"] = stn_df["latitude"].round(6)

stn_df.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,fpath_or_id
0,EIR,Eiriksdal,NaN,6.215740,61.233612,../shapefiles/Eiriksdal.shp
1,GAU,Gautingdalselva,NaN,6.144167,61.241150,../shapefiles/Gautingdalselva.shp
2,HAA,Haaland,NaN,6.077150,61.220030,../shapefiles/Haaland.shp
3,HOY,Hoyanger,NaN,6.073730,61.216397,../shapefiles/Hoyanger.shp


In [6]:
# Build geom
stn_gdf = gpd.GeoDataFrame(
    stn_df,
    crs="epsg:4326",
    geometry=gpd.points_from_xy(stn_df.longitude, stn_df.latitude),
).copy()
stn_gdf["geom"] = stn_gdf["geometry"].apply(lambda x: WKTElement(x.wkt, srid=4326))
del stn_gdf["fpath_or_id"], stn_gdf["geometry"]

stn_gdf.head()

,station_code,station_name,aquamonitor_id,longitude,latitude,geom
0,EIR,Eiriksdal,NaN,6.215740,61.233612,POINT (6.21574 61.233612)
1,GAU,Gautingdalselva,NaN,6.144167,61.241150,POINT (6.144167 61.24115)
2,HAA,Haaland,NaN,6.077150,61.220030,POINT (6.07715 61.22003)
3,HOY,Hoyanger,NaN,6.073730,61.216397,POINT (6.07373 61.216397)


In [7]:
# # Add stations
# stn_gdf.to_sql(
#     "stations",
#     schema="niva",
#     con=eng,
#     if_exists="append",
#     index=False,
#     dtype={"geom": Geometry("POINT", srid=4326)},
#     method="multi",
#     chunksize=1000,
# )

## 3. Add project-stations

In [8]:
# Build table for projects-stations
# Get station IDs
stn_codes = tuple(stn_df["station_code"].unique())
sql = text("SELECT station_id FROM niva.stations WHERE station_code IN :stn_codes")
prst_df = pd.read_sql(sql, params={"stn_codes": stn_codes}, con=eng)

# Get project ID
proj_code = proj_df["project_code"].values[0]
sql = text("SELECT project_id FROM niva.projects WHERE project_code = :proj_code")
proj_id = pd.read_sql(sql, params={"proj_code": str(proj_code)}, con=eng)[
    "project_id"
].iloc[0]

prst_df["project_id"] = proj_id
prst_df.head()

,station_id,project_id
0,1285,6
1,1286,6
2,1287,6
3,1288,6


In [9]:
# # Add projects-stations
# prst_df.to_sql(
#     "projects_stations",
#     schema="niva",
#     con=eng,
#     if_exists="append",
#     index=False,
#     method="multi",
#     chunksize=1000,
# )

## 4. Add catchments

In [10]:
# Loop over catchments
gdf_list = []
for idx, row in stn_df.iterrows():
    stn_code = row["station_code"]
    shp_path = row["fpath_or_id"]
    cat_gdf = gpd.read_file(shp_path)
    cat_gdf["dissolve"] = 1
    cat_gdf["station_code"] = stn_code
    cat_gdf = cat_gdf.dissolve(by="dissolve").reset_index()
    cat_gdf = cat_gdf[["station_code", "geometry"]]
    gdf_list.append(cat_gdf)

cat_gdf = pd.concat(gdf_list).reset_index(drop=True)
cat_gdf.head()

,station_code,geometry
0,EIR,"POLYGON ((37248.834 6823946.164, 37499.337 682..."
1,GAU,"POLYGON ((29139.510 6824275.830, 29235.270 682..."
2,HAA,"POLYGON ((22172.792 6822076.278, 22182.177 682..."
3,HOY,"POLYGON ((29139.510 6824275.830, 29235.270 682..."


In [11]:
# Get station IDs from db
proj_id = 6
sql = text(
    "SELECT station_id, station_code FROM niva.stations "
    "WHERE station_id IN ( "
    "  SELECT station_id from niva.projects_stations "
    "  WHERE project_id = :proj_id)"
)
stn_ids = pd.read_sql(sql, params={"proj_id": proj_id}, con=eng)

# Join catchments
cat_gdf = cat_gdf.merge(stn_ids, on="station_code")

# Reproject to WGS84 GCS
cat_gdf = cat_gdf.to_crs("epsg:4326")

# Cast to multi
cat_gdf["geometry"] = [
    MultiPolygon([feature]) if type(feature) == Polygon else feature
    for feature in cat_gdf["geometry"]
]

# Convert 3D to 2D
cat_gdf["geom"] = cat_gdf["geometry"].apply(
    lambda x: shapely.wkb.loads(shapely.wkb.dumps(x, output_dimension=2))
)

# Tidy
cat_gdf["geom"] = cat_gdf["geom"].apply(lambda x: WKTElement(x.wkt, srid=4326))
cat_gdf = cat_gdf[["station_id", "geom"]]

cat_gdf.head()

,station_id,geom
0,1285,MULTIPOLYGON (((6.352129055221746 61.273364696...
1,1286,MULTIPOLYGON (((6.201841751529703 61.266608234...
2,1287,MULTIPOLYGON (((6.079085860095882 61.238651879...
3,1288,MULTIPOLYGON (((6.201841751529719 61.266608234...


In [12]:
# # Add catchments
# cat_gdf.to_sql(
#     "catchments",
#     schema="niva",
#     con=eng,
#     if_exists="append",
#     index=False,
#     dtype={"geom": Geometry("MULTIPOLYGON", srid=4326)},
#     method="multi",
#     chunksize=1000,
# )